In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the main dataset path (no separate val folder needed!)
dataset_dir = r"C:\Users\sweth\OneDrive\MajorProject"

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    validation_split=0.2  # 20% of images will be used for validation
)

# Load Training Data (80% of the dataset)
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"  # Training set (80%)
)

# Load Validation Data (20% of the dataset)
val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"  # Validation set (20%)
)


Found 3360 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_dir = r"C:\Users\sweth\OneDrive\MajorProject"

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # 80% training, 20% validation
)

# Load Training Data
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

# Load Pretrained MobileNetV2 (without the top layer)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers (to use pretrained features)
base_model.trainable = False

# Add custom layers for TB detection
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)  # Prevent overfitting
output = Dense(1, activation="sigmoid")(x)  # Binary classification (TB vs Normal)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,  # You can increase epochs for better results
    verbose=1
)

# Save the trained model
model.save("tb_detection_mobilenetv2.h5")

print("🎉 Model training complete! Saved as tb_detection_mobilenetv2.h5")


Found 3360 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


C:\Users\sweth\.software\MajorProject\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 87s 756ms/step - accuracy: 0.9354 - loss: 0.1744 - val_accuracy: 0.9964 - val_loss: 0.0213
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 77s 731ms/step - accuracy: 0.9947 - loss: 0.0175 - val_accuracy: 0.9952 - val_loss: 0.0102
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 758ms/step - accuracy: 0.9937 - loss: 0.0206 - val_accuracy: 0.9940 - val_loss: 0.0130
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 81s 770ms/step - accuracy: 0.9998 - loss: 0.0051 - val_accuracy: 0.9976 - val_loss: 0.0063
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 81s 773ms/step - accuracy: 0.9985 - loss: 0.0090 - val_accuracy: 0.9988 - val_loss: 0.0066
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 78s 743ms/step - accuracy: 0.9978 - loss: 0.0055 - val_accuracy: 0.9881 - val_loss: 0.0350
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 81s 774ms/step - accuracy: 0.9968 - loss: 0.0099 - val_accuracy: 0.9976 - val_loss: 0.0076
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 82s 777ms/step - accuracy: 1.0000 - loss: 5

🎉 Model training complete! Saved as tb_detection_mobilenetv2.h5


In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, Label, Button, Canvas, Frame

# Load trained model
model = tf.keras.models.load_model("tb_detection_mobilenetv2.h5")

# Set default image paths (update with actual paths)
BG_IMAGE_MAIN = r"C:\Users\sweth\OneDrive\MajorProject\background(1).jpeg"
BG_IMAGE_PRED = r"C:\Users\sweth\OneDrive\MajorProject\bg-2(1).jpeg"

class TBDetectionApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("TB Detection System")
        self.root.geometry("900x600")

        # Load and display background image for main window
        if os.path.exists(BG_IMAGE_MAIN):
            self.bg_main = Image.open(BG_IMAGE_MAIN).resize((900, 600))
            self.bg_main_photo = ImageTk.PhotoImage(self.bg_main)
            
            self.canvas = Canvas(self.root, width=900, height=600)
            self.canvas.pack(fill="both", expand=True)
            self.canvas.create_image(0, 0, image=self.bg_main_photo, anchor="nw")

            # Prevent garbage collection
            self.root.bg_main_photo = self.bg_main_photo
        else:
            print("Error: Background image for main window not found!")

        # Frame for buttons
        frame = Frame(self.root, bg="white", padx=20, pady=20)
        frame.place(relx=0.5, rely=0.5, anchor="center")

        Label(frame, text="TB Detection System", font=("Arial", 20, "bold"), bg="white", fg="#333").pack(pady=20)
        Button(frame, text="Predict TB", command=self.open_prediction_window, font=("Arial", 14),
               bg="#4CAF50", fg="white", padx=20, pady=10).pack(pady=10)
        Button(frame, text="Exit", command=self.root.quit, font=("Arial", 14),
               bg="#f44336", fg="white", padx=20, pady=10).pack(pady=10)

        self.root.mainloop()

    def open_prediction_window(self):
        self.root.withdraw()  # Hide main window
        self.pred_root = tk.Toplevel()
        self.pred_root.title("TB Detection - Prediction")
        self.pred_root.geometry("900x600")

        # Load and display background image for second window
        if os.path.exists(BG_IMAGE_PRED):
            self.bg_pred = Image.open(BG_IMAGE_PRED).resize((900, 600))
            self.bg_pred_photo = ImageTk.PhotoImage(self.bg_pred)
            
            self.canvas_pred = Canvas(self.pred_root, width=900, height=600)
            self.canvas_pred.pack(fill="both", expand=True)
            self.canvas_pred.create_image(0, 0, image=self.bg_pred_photo, anchor="nw")

            # Prevent garbage collection
            self.pred_root.bg_pred_photo = self.bg_pred_photo
        else:
            print("Error: Background image for second window not found!")

        # Frame for Upload & Prediction
        frame = Frame(self.pred_root, bg="white", padx=20, pady=20)
        frame.place(relx=0.5, rely=0.5, anchor="center")

        Button(frame, text="Upload X-Ray", command=self.predict_tb, font=("Arial", 14),
               bg="#4CAF50", fg="white", padx=20, pady=10).pack(pady=10)

        self.image_label = Label(frame, bg="white")
        self.image_label.pack()

        self.result_label = Label(frame, text="Prediction: ", font=("Arial", 16, "bold"), bg="white", fg="#333")
        self.result_label.pack(pady=20)

        Button(frame, text="Back", command=self.go_back, font=("Arial", 14),
               bg="#FFA500", fg="white", padx=20, pady=10).pack(pady=10)
        Button(frame, text="Exit", command=self.pred_root.destroy, font=("Arial", 14),
               bg="#f44336", fg="white", padx=20, pady=10).pack(pady=10)

    def go_back(self):
        self.pred_root.destroy()
        self.root.deiconify()  # Show main window again

    def predict_tb(self):
        file_path = filedialog.askopenfilename(title="Select Chest X-Ray Image",
                                               filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        if not file_path:
            return

        img = self.preprocess_image(file_path)
        prediction = model.predict(img)
        confidence = prediction[0][0] * 100

        result_text = f"TB Detected ({confidence:.2f}%)" if prediction[0][0] > 0.5 else f"Normal ({100 - confidence:.2f}%)"
        self.result_label.config(text=f"Prediction: {result_text}",
                                 fg="#FF0000" if prediction[0][0] > 0.5 else "#008000")

        self.display_image(file_path)

    def preprocess_image(self, img_path):
        img = Image.open(img_path).convert("RGB")
        img = img.resize((224, 224))
        img = np.array(img) / 255.0
        img = np.expand_dims(img, axis=0)
        return img

    def display_image(self, img_path):
        img = Image.open(img_path).resize((300, 300))
        img = ImageTk.PhotoImage(img)
        self.image_label.config(image=img)
        self.image_label.image = img

if __name__ == "__main__":
    TBDetectionApp()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
